# Solución P3
Comenzamos definiendo la clase para representar árboles de Merkle

In [1]:
class MerkleTree:
    def __init__(self, strings, hash_func):
        self.hash_func = hash_func
        
        # Don't mess with the original variable
        strings = strings.copy()
        
        # Partimos asegurándonos de tener una cantidad par de strings
        if len(strings) % 2:
            strings.append(strings[-1])
        
        # Construiremos el árbol como una lista en la que los hijos del
        # nodo en la posición i están en las posiciones 2i + 1 y 2i + 2.
        # Para esto necesitamos que la cantidad de hojas sea una potencia de 2.
        # Nos aseguramos de esto repitiendo los últimos dos strings tantas veces
        # como sea necesario. x & x - 1 == 0 ocurre sólo para potencias de 2.
        while len(strings) & (len(strings) - 1) != 0:
            strings += [strings[-2], strings[-1]]

        self.tree = []
        prev_level = [hash_func(x) for x in strings]
        
        while len(prev_level) > 1:
            self.tree = prev_level + self.tree
            next_level = []
            for i in range(0, len(prev_level), 2):
                next_level.append(hash_func(prev_level[i] + prev_level[i + 1]))
            prev_level = next_level

        self.tree = prev_level + self.tree
        
    def get_root(self):
        # La raíz está en la posición 0
        return self.tree[0]
    
    def get_proof_for(self, item):
        try:
            next_pos = self.tree.index(self.hash_func(item))
        except ValueError:
            return None        
        
        if next_pos < len(self.tree) // 2:
            # En este caso el item no era parte de la lista original
            return None
        
        proof = []
        while next_pos:
            if next_pos % 2:
                proof.append((self.tree[next_pos + 1], "d"))
            else:
                proof.append((self.tree[next_pos - 1], "i"))
            next_pos = (next_pos - 1) // 2
        return proof

Ahora escribimos la función para verificar una prueba

In [2]:
def verify(root, item, proof, hash_func):
    current_element = hash_func(item)
    for sibling, side in proof:
        if side == "d":
            current_element = hash_func(current_element + sibling)
        else:
            current_element = hash_func(sibling + current_element)
    
    return current_element == root

# Probando la función
Lo que viene a continuación no es parte de la tarea, es sólo una prueba de nuestra solución usando los ejeplos publicados en el [issue de ejemplos](https://github.com/UC-IIC3253/2022/issues/34). Comenzamos definiendo funciones de hash.

In [3]:
import hashlib
def MD5(string):
    return hashlib.md5(string.encode()).hexdigest()

def SHA256(string):
    return hashlib.sha256(string.encode()).hexdigest()

Definimos un primer árbol de Merkle e imprimimos su raíz.

In [4]:
tree = MerkleTree(['asdf', 'wena', '1234', 'hola', 'chao', 'a', 'b', 'c', 'd', 'e'], MD5)
print(tree.get_root())

6cf90496d8de31fb9777408af3aa4c92


Obtenemos una prueba para el string `1234`

In [5]:
proof = tree.get_proof_for('1234')
print(proof)

[('4d186321c1a7f0f354b297e8914ab240', 'd'), ('ad21ab3205307ce250876e1fe2de3f98', 'i'), ('e59133f9f3e8f7421feaa80e8f46598f', 'd'), ('457b539a5afa2646e3b7457e6cac35d7', 'd')]


Verificamos que la prueba sea correcta con la función `verify`

In [6]:
verify(tree.get_root(), '1234', proof, MD5)

True

Generamos una prueba incorrecta sacándole la última tupla a la prueba anterior. Verificamos que no sea una prueba correcta.

In [7]:
wrong_proof = proof[:-1]
verify(tree.get_root(), '1234', wrong_proof, MD5)

False

Hagamos otro árbol de prueba usando SHA-256 y veamos su raíz.

In [8]:
tree = MerkleTree(['IIC3253', 'es', 'el', 'mejor', 'curso', 'y', 'lo', 'sabes'], SHA256)
tree.get_root()

'366b9d1289b2e34c466374f840f1ef6c40ecd352c422a4cfca8bc276293b954c'

Obtenemos una prueba para el string `y` y otra para el string `el`

In [9]:
proof = tree.get_proof_for('y')
print(proof)
proof2 = tree.get_proof_for('el')
print(proof2)

[('a4a670183ad8c9972bce2f85fcefbd0e15118f3310bb6dbd217b847a7578ff17', 'i'), ('2d26e5d60e3a43424820af5a027f0a4f474a3bd1783adbcf2c8af237a1f23acf', 'd'), ('7b138c7c9f60b91765dc2452a6152090a9b31433e0a103529eb6f1c473ca8e47', 'i')]
[('a9222478217b151579d5f0b500093fa0e636104b14c42f0fde379ea65ad69159', 'd'), ('837c761ca2be72cddd36e1770bb07e23739a313564035097e1368676795cd5da', 'i'), ('d62c0418530d66d60615f0ec2a2bf4ee840e1538c34333a090a1f0e509d59dc4', 'd')]


Finalmente verificamos las pruebas, y vemos que cada prueba corresponde a su propio string.

In [10]:
print(verify(tree.get_root(), 'y', proof, SHA256))
print(verify(tree.get_root(), 'y', proof2, SHA256))
print(verify(tree.get_root(), 'el', proof2, SHA256))
print(verify(tree.get_root(), 'el', proof, SHA256))

True
False
True
False


# Generando los ejemplos
A continuación está el código utilizado para generar los ejemplos de corrección.
Verificaremos cada una de las funcionalidades usando distintos niveles de complejidad.

In [11]:
import uuid
import json
import random

# Los elementos que serán utilizados para evaluar
trees = []

# Usaremos las funciones de hash aleatoriamente
hash_funcs = {'sha256': SHA256, 'md5': MD5}

Definiremos árboles con distintas cantidades de hojas:
 - Potencias de 2
 - Potencias de 2 menos 1
 - Potencias de 2 menos 2
 - Potencias de 2 más 1
 - Potencias de 2 más 2

In [12]:
leaf_amounts = [30, 31, 32, 33, 34, 62, 63, 64, 126, 127, 128, 129, 130]

for amount in leaf_amounts:
    # Generamos las hojas y el árbol
    leafs = [str(uuid.uuid4()) for i in range(amount)]
    hash_func_name = random.choice(list(hash_funcs.keys()))
    hash_func = hash_funcs[hash_func_name]
    tree = MerkleTree(leafs, hash_func)
    root = tree.get_root()
    
    # Generaremos pruebas para 3 strings aleatorios que sí corresponden al árbol.
    proofs = []
    for i in range (3):
        leaf = random.choice(leafs)
        proof = tree.get_proof_for(leaf)
        proofs.append({'node': leaf, 'proof': proof})
        assert verify(root, leaf, proof, hash_func)
        
    # Una prueba para un nodo que no está
    proofs.append({
        'node': 'A very random string', 'proof': None
    })

    # Otra para un nodo cuyo hash está en el árbol pero no es parte de las hojas
    index = random.randint(0, len(leafs) - 1)
    index += index % 2
    proofs.append({
        'node': tree.tree[index - 1] + tree.tree[index], 'proof': None
    })
    
    # Finalmente un caso bien borde
    proofs.append({
        'node': tree.tree[-1] + tree.tree[-2], 'proof': None
    })
    
    
    # esto será exportado para evaluar que el código genere exactamente la misma
    # raíz y las mismas pruebas para cada uno de los ejemplos
    trees.append({
        'leafs': leafs,
        'root': root,
        'proofs': proofs,
        'hash_func': hash_func_name
    })

Exportemos el resultado a un archivo json

In [13]:
output = open('grading_trees.json', 'w')
output.write(json.dumps({'trees': trees}, indent=2))
output.close()

# Evaluación
A continuación se muestra el código para obtener el puntaje dada una clase `MerkleTree` y una función `verify`

In [14]:
def get_points(MerkleTreeClass, verify_func):
    examples = json.loads(open('grading_trees.json').read())
    
    # Variables para contar lo evaluado y cuántas cosas fueron correctas
    correct_trees = 0
    correct_proofs = 0
    correct_verifications = 0
    correct_false_verifications = 0
    total_proofs = 0
    total_verifications = 0
    
    # Evaluamos contra cada uno de los ejemplos
    for example in examples['trees']:
        
        # Sacamos la función de hash correspondiente
        hash_func = hash_funcs[example['hash_func']]
        
        # Verificamos que se genere el árbol con la raíz correcta
        try:
            tree = MerkleTreeClass(example['leafs'], hash_func)
            root = tree.get_root()
            if root == example['root']:
                correct_trees += 1
        except:
            tree = None
        
        for example_proof in example['proofs']:
            total_proofs += 1
            
            # Verificamos que se genere la prueba correcta. JSON no admite tuplas,
            # por lo que usa listas de listas. Debemos convertirlas :(
            if tree is not None:
                try:
                    proof = tree.get_proof_for(example_proof['node'])
                    if proof is None and example_proof['proof'] is None:
                        correct_proofs += 1
                    elif proof == [tuple(i) for i in example_proof['proof']]:
                        correct_proofs += 1
                except:
                    pass
                
            # Vemos que la verificación se haga correctamente
            if example_proof['proof'] is not None:
                total_verifications += 1
                
                try:
                    if verify(example['root'], example_proof['node'], example_proof['proof'], hash_func):
                        correct_verifications += 1

                    if not verify(example['root'], example_proof['node'], example_proof['proof'][1:], hash_func):
                        correct_false_verifications += 1
                except: pass
                    
    trees_points = 2 * correct_trees / len(examples["trees"])
    proofs_points = 1.5 * correct_proofs / total_proofs
    verification_points = 1.5 * correct_verifications / total_verifications
    false_verification_points = correct_false_verifications / total_verifications
                    
    print(f'Árboles correctos: {correct_trees} / {len(examples["trees"])}')
    print(f'{trees_points} / 2.0 puntos\n')
    
    print(f'Pruebas correctas: {correct_proofs} / {total_proofs}')
    print(f'{proofs_points} / 1.5 puntos\n')
    
    print(f'Verificaciones correctas: {correct_verifications} / {total_verifications}')
    print(f'{verification_points} / 1.5 puntos\n')
    
    print(f'Verificaciones falsas correctas: {correct_false_verifications} / {total_verifications}')
    print(f'{false_verification_points} / 1.0 puntos\n')
    
    total_points = trees_points + proofs_points + verification_points + false_verification_points
    print(f'{total_points} / 6.0 puntos')
    
    return total_points

Probamos la función con la clase y función programadas:

In [15]:
points = get_points(MerkleTree, verify)
grade = 1 + points
print(f'\nNos sacamos un {grade}!')

Árboles correctos: 13 / 13
2.0 / 2.0 puntos

Pruebas correctas: 78 / 78
1.5 / 1.5 puntos

Verificaciones correctas: 39 / 39
1.5 / 1.5 puntos

Verificaciones falsas correctas: 39 / 39
1.0 / 1.0 puntos

6.0 / 6.0 puntos

Nos sacamos un 7.0!
